In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
from tensorflow import keras
from matplotlib import pyplot
import imblearn
import gensim
import nltk
from pathlib import Path
import time
import tensorflow_hub as hub

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
import pydot

In [3]:
df_date = pd.read_csv('/storage/data/ad_matched_queries_20190831__by_date.csv')
df_date.head(5)

,campaign_label,date,source_medium,campaign,ad_group,ad_matched_query,ad_cost,impressions,clicks,users,new_users,sessions,bounces,created_accounts,transactions,transaction_revenue
0,brand,2017-03-07,google / cpc,Brand Terms,Paperspace For Teams,paper space,0.31,4,1,1,1,1,0,0,0,0.0
1,brand,2017-03-07,google / cpc,Brand Terms,Paperspace For Teams,paperspace,1.06,28,5,4,3,4,2,0,0,0.0
2,brand,2017-03-08,google / cpc,Brand Terms,Paperspace For Teams,paperspace,0.94,24,3,1,0,1,0,0,0,0.0
3,brand,2017-03-09,google / cpc,Brand Terms,Paperspace For Teams,paperspace,1.70,50,4,2,2,2,1,0,0,0.0
4,brand,2017-03-10,google / cpc,Brand Terms,Paperspace For Teams,paperspace,4.57,49,7,5,4,9,0,1,1,6.0


In [4]:
REPORT_SOURCE = 'google/ads'
REPORT_ID = 'InsightFall2019 Ad Group Keyword 20191001'
FILENAME = 'aggregate_20190831.csv.gz'

PATH_DATA = Path('/') / 'storage' / 'data' / REPORT_SOURCE / REPORT_ID / FILENAME
df_cat = pd.read_csv(PATH_DATA)
df_cat.head(5)


,Campaign ID,Ad group ID,Keyword ID,Campaign,Ad group,Search keyword,Keyword/Placement,Search keyword match type,Campaign type,Campaign bid strategy type,...,Currency,Cost,Impressions,Clicks,Conversions,Conv. value,Avg. position,Impr. (Abs. Top) %,Impr. (Top) %,Search Impr. share
0,765732120,39460828726,44160165835,General Product Terms,Cloud Virtual Desktop - Enterprise Market,+Cloud +Virtual +Desktop,+Cloud +Virtual +Desktop,Broad,Search Only,Target CPA,...,USD,0.0,4,0,0.0,0.0,1.5,0.0,0.0,0.3636
1,765732120,39616922879,341042909967,General Product Terms,VDI Cloud,+vdi in the +cloud,+vdi in the +cloud,Broad,Search Only,Target CPA,...,USD,0.0,1,0,0.0,0.0,7.0,0.0,0.0,1.0000
2,765732120,43344958467,27994524387,General Product Terms,Cloud Virtual Machine - Enterprise Market,+Cloud +Virtual +Machine,+Cloud +Virtual +Machine,Broad,Search Only,Target CPA,...,USD,0.0,2,0,0.0,0.0,2.0,0.0,0.0,1.0000
3,766802375,39100138983,28912106981,Competitor Terms,VMware Cloud Computing,+cloud +vmware,+cloud +vmware,Broad,Search Only,Enhanced CPC,...,USD,0.0,1,0,0.0,0.0,3.0,0.0,0.0,0.5000
4,765732120,39616922879,32957936454,General Product Terms,VDI Cloud,+vdi +cloud,+vdi +cloud,Broad,Search Only,Target CPA,...,USD,0.0,3,0,0.0,0.0,1.0,0.0,0.0,1.0000


In [5]:
df_cat.rename(columns={'Ad group':'ad_group'}, inplace=True)
df_cat1 = df_cat[['ad_group','Campaign', 'Search keyword', 'Search keyword match type', 'Campaign type', 'Campaign bid strategy type']].copy()
df_cat1.head(5)

,ad_group,Campaign,Search keyword,Search keyword match type,Campaign type,Campaign bid strategy type
0,Cloud Virtual Desktop - Enterprise Market,General Product Terms,+Cloud +Virtual +Desktop,Broad,Search Only,Target CPA
1,VDI Cloud,General Product Terms,+vdi in the +cloud,Broad,Search Only,Target CPA
2,Cloud Virtual Machine - Enterprise Market,General Product Terms,+Cloud +Virtual +Machine,Broad,Search Only,Target CPA
3,VMware Cloud Computing,Competitor Terms,+cloud +vmware,Broad,Search Only,Enhanced CPC
4,VDI Cloud,General Product Terms,+vdi +cloud,Broad,Search Only,Target CPA


In [6]:
df_searchwords = df_cat1[df_cat1['ad_group'] == 'VDI Cloud']
#print(df_searchwords.head(100))
print(df_searchwords['Search keyword'].unique())

['+vdi in the +cloud' '+vdi +cloud' '+cloud +vdi' 'vdi in the cloud'
 '+vdi +in +cloud' '+vdi +on +cloud']


In [7]:
df_unique = df_cat.groupby(['ad_group','Search keyword', 'Search keyword match type']).size().reset_index().rename(columns={0:'count'})
df_unique.head(25)

,ad_group,Search keyword,Search keyword match type,count
0,3ds Max,+3ds +max +mac +os,Broad,4
1,3ds Max,+3ds +max +mac +version,Broad,1
2,3ds Max,+autodesk +3ds +max +mac,Broad,14
3,3ds Max,+mac +3ds +max,Broad,66
4,3ds Max,3ds max for mac,Exact,57
5,3ds Max,3ds max for mac os,Exact,1
6,3ds Max,3ds max mac,Exact,50
7,3ds Max,3ds max mac os x,Exact,2
8,3ds Max,autodesk 3ds max for mac,Exact,6
9,AI Cloud Amazon_broad,+amazon +cloud +ai,Broad,78


In [8]:
srch_keywords_dict = {}
for name, group in df_unique.groupby('ad_group'):
    values = (' '.join(map(str, group['Search keyword'].values)))
    unique_values = ' '.join(list(set(values.split(' '))))
    srch_keywords_dict[name] = unique_values

srch_keywords_df = pd.DataFrame.from_dict(srch_keywords_dict, orient='index').reset_index()
srch_keywords_df.rename(columns={'index': "ad_group", 0: "srch_keywords"}, inplace=True)

print(srch_keywords_df.shape)
filtered_df = srch_keywords_df[srch_keywords_df['ad_group'].str.endswith('_broad') | srch_keywords_df['ad_group'].str.endswith('_exact') | srch_keywords_df['ad_group'].str.endswith('_phrase')].copy()
print(filtered_df)


#use campaign as another variable to split the stuff on
#we are filtering out 1/3 of the data because we can't join on type of match

#IDEAL
#match specific set of search keywords within ad group that were used for the ad_matched_query
#pulling in the ad group id or keyword id into ad_match_queries dataset

#TODO
#create embedding of ad_match_query
#create embedding of search keyword, one hot encode match type

(386, 2)
                          ad_group  \
1            AI Cloud Amazon_broad   
2            AI Cloud Google_broad   
3               AI Cloud Gpu_broad   
4         AI Cloud Microsoft_broad   
5                   AI Cloud_broad   
..                             ...   
381          Windows On Mac_phrase   
382  Windows Programs On Mac_exact   
383       Windows VM On Mac_phrase   
384  iPython Notebook Online_exact   
385  iPython Notebook Server_exact   

                                         srch_keywords  
1    +aws +ai +intelligence +artificial +amazon +cloud  
2         +google +ai +intelligence +artificial +cloud  
3                                      +ai +gpu +cloud  
4      +microsoft +ai +intelligence +artificial +cloud  
5                 +ai +intelligence +artificial +cloud  
..                                                 ...  
381                                 windows for on mac  
382                        programs on windows mac run  
383                  

In [9]:
df_join = pd.merge(filtered_df, df_date, on='ad_group')

df_join.head()

,ad_group,srch_keywords,campaign_label,date,source_medium,campaign,ad_matched_query,ad_cost,impressions,clicks,users,new_users,sessions,bounces,created_accounts,transactions,transaction_revenue
0,AI Cloud Amazon_broad,+aws +ai +intelligence +artificial +amazon +cloud,ml,2018-11-29,google / cpc,2018_NA-US_en_ML_search_desktop,amazon cloud ai,2.49,1,1,1,0,1,1,0,0,0.0
1,AI Cloud Google_broad,+google +ai +intelligence +artificial +cloud,ml,2018-10-01,google / cpc,2018_NA-US_en_ML_search_desktop,google cloud ai client,19.79,2,1,1,1,1,1,0,0,0.0
2,AI Cloud Google_broad,+google +ai +intelligence +artificial +cloud,ml,2018-10-02,google / cpc,2018_NA-US_en_ML_search_desktop,google cloud ai,15.00,18,1,1,1,1,1,0,0,0.0
3,AI Cloud Google_broad,+google +ai +intelligence +artificial +cloud,ml,2018-10-04,google / cpc,2018_NA-US_en_ML_search_desktop,google cloud ai,0.00,1,0,0,0,0,0,0,0,0.0
4,AI Cloud Google_broad,+google +ai +intelligence +artificial +cloud,ml,2018-10-22,google / cpc,2018_EUR-W-CH_en_ML_search_desktop,google cloud artificial intelligence,2.43,1,1,0,0,0,0,0,0,0.0


In [15]:
#click_counts = df_join['clicks'].value_counts()
df_join.loc[df_join['clicks'] < 1, 'clicks'] = 0
df_join.loc[df_join['clicks'] >= 1, 'clicks'] = 1
df_join['clicks']
clicks = df_join['clicks']
clicks.to_pickle("clicks.pkl")
clicks.shape

(165051,)

In [11]:
#corpus: combine keywords and queries
#df_join['corpus'] = df_join["srch_keywords"].map(str) + ' ' + df_join["ad_matched_query"].map(str)
#df_join['corpus'] = df_join['corpus'].apply(lambda x: ' '.join(pd.unique(x.split())))
#df_join['corpus'].head(5)

In [12]:
#create match type column
df_join['match_type'] = np.nan
df_join['match_type'] = pd.np.where(df_join['ad_group'].str.contains("broad"), "broad",
                        pd.np.where(df_join['ad_group'].str.contains("exact"), "exact",
                        pd.np.where(df_join['ad_group'].str.contains("phrase"), "phrase", 'ERROR')))
df_join.match_type.unique()
df_join.head(5)

,ad_group,srch_keywords,campaign_label,date,source_medium,campaign,ad_matched_query,ad_cost,impressions,clicks,users,new_users,sessions,bounces,created_accounts,transactions,transaction_revenue,match_type
0,AI Cloud Amazon_broad,+aws +ai +intelligence +artificial +amazon +cloud,ml,2018-11-29,google / cpc,2018_NA-US_en_ML_search_desktop,amazon cloud ai,2.49,1,1,1,0,1,1,0,0,0.0,broad
1,AI Cloud Google_broad,+google +ai +intelligence +artificial +cloud,ml,2018-10-01,google / cpc,2018_NA-US_en_ML_search_desktop,google cloud ai client,19.79,2,1,1,1,1,1,0,0,0.0,broad
2,AI Cloud Google_broad,+google +ai +intelligence +artificial +cloud,ml,2018-10-02,google / cpc,2018_NA-US_en_ML_search_desktop,google cloud ai,15.00,18,1,1,1,1,1,0,0,0.0,broad
3,AI Cloud Google_broad,+google +ai +intelligence +artificial +cloud,ml,2018-10-04,google / cpc,2018_NA-US_en_ML_search_desktop,google cloud ai,0.00,1,0,0,0,0,0,0,0,0.0,broad
4,AI Cloud Google_broad,+google +ai +intelligence +artificial +cloud,ml,2018-10-22,google / cpc,2018_EUR-W-CH_en_ML_search_desktop,google cloud artificial intelligence,2.43,1,1,0,0,0,0,0,0,0.0,broad


In [13]:
df_meta = pd.DataFrame(df_join, columns = ['match_type', 'date']) 
df_meta.head(5)

,match_type,date
0,broad,2018-11-29
1,broad,2018-10-01
2,broad,2018-10-02
3,broad,2018-10-04
4,broad,2018-10-22


In [14]:
encoded_columns = pd.get_dummies(df_meta['match_type'])
df_meta = df_meta.join(encoded_columns).drop('match_type', axis=1)
df_meta.head(5)

,date,broad,exact,phrase
0,2018-11-29,1,0,0
1,2018-10-01,1,0,0
2,2018-10-02,1,0,0
3,2018-10-04,1,0,0
4,2018-10-22,1,0,0


In [15]:
# date preprocessing

from datetime import date
from sklearn.preprocessing import LabelEncoder 

#Transform string to date
df_meta['date'] = pd.to_datetime(df_meta['date'])

#Extracting Year
df_meta['year'] = df_meta['date'].dt.year

#Extracting Month
df_meta['month'] = df_meta['date'].dt.month

#Extracting Day
df_meta['day'] = df_meta['date'].dt.day

#Extracting the weekday name of the date
df_meta['day_name'] = df_meta['date'].dt.day_name()

#Extracting passed years since the date
df_meta['passed_years'] = date.today().year - df_meta['date'].dt.year

#Extracting passed months since the date
df_meta['passed_months'] = (date.today().year - df_meta['date'].dt.year) * 12 + date.today().month - df_meta['date'].dt.month

#normalize date as float
df_meta['date'] = pd.to_datetime(df_meta['date']).astype('int64')
max_a = df_meta.date.max()
min_a = df_meta.date.min()
min_norm = -1
max_norm = 1
df_meta['norm_date'] = (df_meta.date- min_a) *(max_norm - min_norm) / (max_a-min_a) + min_norm
df_meta['norm_date'].astype(float)
df_meta['norm_date'].head(5)

df_meta = df_meta.drop(columns=['date'], axis=1)

le = LabelEncoder()
df_meta['day_name'] = le.fit_transform(df_meta['day_name'])

df_meta.head(5)

,broad,exact,phrase,year,month,day,day_name,passed_years,passed_months,norm_date
0,1,0,0,2018,11,29,4,1,11,0.271523
1,1,0,0,2018,10,1,1,1,12,0.115232
2,1,0,0,2018,10,2,5,1,12,0.117881
3,1,0,0,2018,10,4,4,1,12,0.123179
4,1,0,0,2018,10,22,1,1,12,0.170861


In [16]:
meta = df_meta
meta.to_pickle("metadata.pkl")

In [17]:
#from sklearn import preprocessing

#scaler = preprocessing.StandardScaler()# Fit your data on the scaler object

#df_join[['ad_cost']] = scaler.fit_transform(df_join[['ad_cost']])

In [18]:
# clean keywords
from nltk.stem import PorterStemmer
df_join['srch_keywords'].str.lower().str.replace('[!"#$%&()*+-/:;<=>?@[\\]^_`{|}~]',' ')
df_join['srch_keywords'].str.replace("[0-9]", ' ')
df_join['srch_keywords'].dropna()
df_join['srch_keywords'] = df_join['srch_keywords'].apply(lambda x:' '.join(x.split()))
#unique_words = df_append.unique()

df_join['srch_keywords'] = df_join['srch_keywords'].str.replace('+','')

ps = PorterStemmer()
df_join['srch_keywords'].map(ps.stem)

print(df_join['srch_keywords'].shape)
srch_keywords = df_join['srch_keywords']
df_join['srch_keywords'].head()

(165051,)


0    aws amazon cloud intelligence artificial ai
1        google cloud intelligence artificial ai
2        google cloud intelligence artificial ai
3        google cloud intelligence artificial ai
4        google cloud intelligence artificial ai
Name: srch_keywords, dtype: object

In [19]:
# clean queries
from nltk.stem import PorterStemmer
df_join["ad_matched_query"].str.lower().str.replace('[!"#$%&()*+-/:;<=>?@[\\]^_`{|}~]',' ')
df_join["ad_matched_query"].str.replace("[0-9]", ' ')
df_join["ad_matched_query"].dropna()
df_join["ad_matched_query"] = df_join["ad_matched_query"].apply(lambda x:' '.join(x.split()))
#unique_words = df_append.unique()

df_join["ad_matched_query"] = df_join["ad_matched_query"].str.replace('+','')

ps = PorterStemmer()
df_join["ad_matched_query"].map(ps.stem)
df_join["ad_matched_query"].head()
ad_matched_query = df_join["ad_matched_query"]
# tokenize by letter
#df_date['stemmed'] = df_date['ad_matched_query'].apply(lambda x: [stemmer.stem(x) for y in x])

In [20]:
#load ELMo from tensorflow hub
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [21]:
# test
tokens_input = [["the", "cat", "is", "on", "the", "mat"],
                ["dogs", "are", "in", "the", "fog", ""]]
tokens_length = [6, 5]
embeddings = elmo(
    inputs={
        "tokens": tokens_input,
        "sequence_len": tokens_length
    },
    signature="tokens",
    as_dict=True)["elmo"]
embeddings.shape

TensorShape([Dimension(2), Dimension(6), Dimension(1024)])

In [22]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]
  #np.save('embeddings.pkl', embeddings)   
  print(embeddings)
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [23]:
import pickle

#list_keywords1= [srch_keywords[i:i+1000] for i in range(0,80000,1000)]
#elmo_keywords1 = [elmo_vectors(x) for x in list_keywords1]
#elmo_keywords_array1 = np.concatenate(elmo_keywords1, axis = 0)

#pickle_out = open("elmo_keywords_array1.pickle","wb")
#pickle.dump(elmo_keywords_array1, pickle_out)
#pickle_out.close()

In [24]:
#list_keywords2 = [srch_keywords[i:i+1000] for i in range(80000,srch_keywords.shape[0],1000)]
#elmo_keywords2 = [elmo_vectors(x) for x in list_keywords2]
#elmo_keywords_array2 = np.concatenate(elmo_keywords2, axis = 0)

#pickle_out = open("elmo_keywords_array2.pickle","wb")
#pickle.dump(elmo_keywords_array2, pickle_out)
#pickle_out.close()

In [25]:
#list_queries1 = [ad_matched_query[i:i+1000] for i in range(0,80000,1000)]
#elmo_queries1 = [elmo_vectors(x) for x in list_queries1]
#elmo_queries_array1 = np.concatenate(elmo_queries1, axis = 0)

#pickle_out = open("elmo_queries_array1.pickle","wb")
#pickle.dump(elmo_queries_array1, pickle_out)
#pickle_out.close()

In [26]:
#list_queries2 = [ad_matched_query[i:i+1000] for i in range(80000,ad_matched_query.shape[0],1000)]
#elmo_queries2 = [elmo_vectors(x) for x in list_queries2]
#elmo_queries_array2 = np.concatenate(elmo_queries2, axis = 0)

#pickle_out = open("elmo_queries_array2.pickle","wb")
#pickle.dump(elmo_queries_array2, pickle_out)
#pickle_out.close()

In [27]:
#load EMLo arrays
import pickle 

pickle_in = open("elmo_keywords_array1.pickle", "rb")
elmo_keywords_array1 = pickle.load(pickle_in)

pickle_in = open("elmo_keywords_array2.pickle", "rb")
elmo_keywords_array2 = pickle.load(pickle_in)

pickle_in = open("elmo_queries_array1.pickle", "rb")
elmo_queries_array1 = pickle.load(pickle_in)

pickle_in = open("elmo_queries_array2.pickle", "rb")
elmo_queries_array2 = pickle.load(pickle_in)

In [28]:
keywords = np.concatenate((elmo_keywords_array1, elmo_keywords_array2))
keywords.shape
pickle_out = open("elmo_keywords.pickle","wb")
pickle.dump(keywords, pickle_out)
pickle_out.close()

In [29]:
queries = np.concatenate((elmo_queries_array1, elmo_queries_array2))
queries.shape
pickle_out = open("elmo_queries.pickle","wb")
pickle.dump(queries, pickle_out)
pickle_out.close()

In [30]:
pickle_in = open("elmo_keywords.pickle", "rb")
elmo_keywords = pickle.load(pickle_in)

pickle_in = open("elmo_queries.pickle", "rb")
elmo_queries = pickle.load(pickle_in)

In [31]:
both_texts = np.concatenate((elmo_keywords,elmo_queries),axis=1)
df_texts = pd.DataFrame(both_texts)
df_features = pd.concat([df_texts, df_meta], axis=1)

In [32]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_features, clicks,test_size=0.2, random_state=42)
print(x_train.shape,y_train.shape)
#print(X_test.shape,Y_test.shape)
#padded_sequences

(132040, 2058) (132040,)


In [33]:
x_train.head(5)

,0,1,2,3,4,5,6,7,8,9,...,broad,exact,phrase,year,month,day,day_name,passed_years,passed_months,norm_date
157407,0.092828,0.409479,0.064707,-0.090124,-0.014111,-0.294523,0.088621,0.221808,-0.299878,-0.451160,...,0,0,1,2018,12,28,0,1,10,0.348344
20040,-0.007032,0.321585,-0.033830,-0.090664,0.406839,0.127357,0.027218,0.350332,0.047294,-0.107674,...,1,0,0,2019,6,16,3,0,4,0.798675
34387,0.215442,0.246529,0.019714,0.144392,0.296803,-0.134385,-0.002052,0.121719,0.220845,-0.352039,...,0,0,1,2018,11,12,1,1,11,0.226490
110646,0.522093,0.240848,0.166097,-0.176110,0.745745,0.212327,0.349806,0.547386,-0.360240,-0.327405,...,0,1,0,2017,12,28,4,2,22,-0.618543
155584,0.110374,0.088423,0.070693,-0.042047,-0.120731,-0.113762,0.106253,0.148949,-0.080463,-0.249856,...,0,0,1,2018,11,12,1,1,11,0.226490


In [34]:
x_train_meta = x_train.iloc[: , 2048:2058]
x_test_meta = x_test.iloc[: , 2048:2058]

x_train_meta.shape

(132040, 10)

In [35]:
x_train_keywords = x_train.iloc[: , 0:1024]
x_test_keywords = x_test.iloc[: , 0:1024]
x_train_keywords.shape

(132040, 1024)

In [36]:
x_train_queries = x_train.iloc[: , 1024:2048]
x_test_queries = x_test.iloc[: , 1024:2048]
x_train_queries.shape

(132040, 1024)

In [37]:
# pad embedding layer hyperparameters 

from tensorflow.keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence

max_length = len(max(elmo_keywords, key=len))

#padded_keywords = sequence.pad_sequences(keywords, maxlen=max_length)
#print(padded_keywords.shape)

t = Tokenizer(num_words=20000)
t.fit_on_texts(srch_keywords)
vocab_size_keywords = len(t.word_index) + 1
print(vocab_size_keywords)

t = Tokenizer(num_words=20000)
t.fit_on_texts(ad_matched_query)
vocab_size_queries = len(t.word_index) + 1
print(vocab_size_queries)

max_length = 1024

#print(t.document_count)
#print(t.word_index)
#print(t.word_counts)

#encoded_queries = t.texts_to_matrix(df_BYDATE['ad_matched_query'], mode='binary')
#print(encoded_queries)

127
7499


In [38]:
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras import layers
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

In [39]:
input_1 = layers.Input(shape=(1024,)) #, dtype="string"
input_2 = layers.Input(shape=(1024,))
input_3 = Input(shape=(10,))

W1017 00:25:49.527928 140555551729472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1017 00:25:49.529354 140555551729472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [40]:
embedding_layer_keywords = Embedding(vocab_size_keywords, 100, trainable=False)(input_1)
LSTM_Layer_keywords = LSTM(128)(embedding_layer_keywords)

W1017 00:25:49.540491 140555551729472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [41]:
embedding_layer_queries = Embedding(vocab_size_queries, 100, trainable=False)(input_2)
LSTM_Layer_queries = LSTM(128)(embedding_layer_queries)

In [42]:
dense_layer_1 = Dense(10, activation='relu')(input_3)
dense_meta = Dense(10, activation='relu')(dense_layer_1)

In [43]:
concat_layer = Concatenate()([LSTM_Layer_keywords, LSTM_Layer_queries, dense_meta])
dense_layer_3 = Dense(10, activation='relu')(concat_layer)
output = Dense(1, activation='sigmoid')(dense_layer_3)
model = Model(inputs=[input_1, input_2, input_3], outputs=output)

In [44]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

W1017 00:25:50.231182 140555551729472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1017 00:25:50.253283 140555551729472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W1017 00:25:50.257999 140555551729472 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1024, 100)    12700       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_

In [ ]:
from keras.utils.vis_utils import plot_model
import path
import graphviz
from keras.utils.vis_utils import model_to_dot

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
!pip install pydot

In [45]:
history = model.fit(x=[x_train_keywords, x_train_queries, x_train_meta],\
                    y=y_train, batch_size=500, epochs=5, verbose=1, validation_split=0.2)

W1017 00:26:08.199866 140555551729472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 105632 samples, validate on 26408 samples
Epoch 1/5
105632/105632 [==============================] - 859s 8ms/step - loss: 0.7054 - acc: 0.5362 - val_loss: 0.6885 - val_acc: 0.5440
Epoch 2/5
105632/105632 [==============================] - 912s 9ms/step - loss: 0.6935 - acc: 0.5424 - val_loss: 0.6889 - val_acc: 0.5543
Epoch 3/5
105632/105632 [==============================] - 946s 9ms/step - loss: 0.6947 - acc: 0.5405 - val_loss: 0.7179 - val_acc: 0.5538
Epoch 4/5
105632/105632 [==============================] - 878s 8ms/step - loss: 0.6919 - acc: 0.5467 - val_loss: 0.7283 - val_acc: 0.4462
Epoch 5/5
105632/105632 [==============================] - 851s 8ms/step - loss: 0.6933 - acc: 0.5427 - val_loss: 0.6868 - val_acc: 0.5538


In [ ]:
#TFIDF vectorize queries
#from sklearn.feature_extraction.text import TfidfVectorizer
#corpus = [line.split(',') for line in df_BYDATE['ad_matched_query']]
#n_features = 2500
#vectorizer = TfidfVectorizer(ngram_range=(1, 2),stop_words='english', lowercase=True ,max_df=0.8, min_df=10, max_features=n_features)
#x = vectorizer.fit_transform(df_join['ad_matched_query'])
#vocab = vectorizer.vocabulary_
#print(vocab)
#print(vectorizer.idf_)
##df_join['queries_vect']=list(x.toarray())
#print(x.shape)

#corpus_array


In [ ]:
#pd.to_pickle(x, 'TFIDF_vec.pickle')

In [ ]:
#pd.to_pickle(df_join, 'df_non_text_features.pickle')

In [ ]:
print(fit_features.shape)
pd.to_pickle(fit_features, 'fit_features.pickle')